In [27]:
import math
import time
import random
import numpy as np
from cmath import exp
from numpy.fft import fft
from numpy.fft import fftfreq
from scipy.integrate import quad

# Financial Data Analysis via Non Uniform Fourier Transform
## Authors 
* Imane El Bouzid : imane.elbouzid@student-cs.fr
* Richard John Lin : richardjohn.lin@student-cs.fr
## Malliavin Mancino Covariance Estimator
The Malliavin Mancino estimator is a non parametric covariance and volatility estimator that does not rely on the assumption of asset data synchronacy. The main idea behind it is to estimate the covariance matrix's Fourier coefficients by using the asset log-prices' Fourier coefficients thanks to the Bohr convolution formula. Indeed, if we denote by $dp_{i}$ the asset $S_{i}$'s log-price and $\mathscr{F}(f)(k)$ the Fourier coeffecient of order $k$ of the function $f$ :
\begin{equation*}
    \frac{1}{2 \pi} \mathscr{F}\left(\Sigma^{i, j}\right)=\mathscr{F}\left(d p^{i}\right) * \mathscr{F}\left(d p^{j}\right),
\end{equation*}
where : 
\begin{equation*}
    \left(\mathscr{F}\left(d p^{i}\right) * \mathscr{F}\left(d p^{j}\right)\right)(k):=\lim _{N \rightarrow \infty} \frac{1}{2 N+1} \sum_{|s| \leq N} \mathscr{F}\left(d p_{i}\right)(s) \mathscr{F}\left(d p_{j}\right)(k-s)
\end{equation*}
The initial complexity of the Malliavin Mancino estimator as provided in [A Fourier transform for Non Parametric Estimation of Multivariate Volatility](https://projecteuclid.org/journals/annals-of-statistics/volume-37/issue-4/A-Fourier-transform-method-for-nonparametric-estimation-of-multivariate-volatility/10.1214/08-AOS633.f) is $O(N^{2})$. This project's aim is to implement more efficient computation algorithms using the Non Uniform Fourier Transform and to benchmark the coded algorithms regarding their speed and accuracy.

**Authors :** 
* Imane El Bouzid imane.elbouzid@student-cs.fr
* Richard John Lin : richardjohn.lin@student-cs.fr

The Malliavin Mancino covariance estimator relies on the computation of the Fourier coefficients of the log-prices. In this notebook, we implement different estimation methods relying on the Fourier transform, particularly the Non Uniform Fourier Transform. The algorithms of these different methods can be found in the article [*Malliavin-Mancino estimators implemented with non-uniform fast Fourier transforms*](https://arxiv.org/pdf/2003.02842.pdf).

# 1. Uniform Fourier Transform Algorithms

## 1.1. For Loop Algorithm

The For Loop method is the initial computation proposed by Malliavin and Mancino for their estimator. Its complexity is $O(N^{2})$ where N is the total number of observation times. This high computational cost renders the algorithm less than efficient for use on big data sets.

In [3]:
def loop(time_vect_res, price_vect, cutoff_freq = math.inf) :
    """Compute the Fourier coefficients a given price vector for a rescaled time vector
       Returns the results as a dictionnary
       time_vect_res : rescaled time vector, sorted in ascending order
       price_vect : price vector of a product with respect to the time vector not log(price)
       cutoff_freq : determines the number of Fourier coefficients computed (2N+1). By default, set to Nyquist frequency
    """
    cutoff_freq = min(cutoff_freq, Nyquist_freq(time_vect_res))
    n = len(time_vect_res)
    dico_coeffs = {}
    dico_delta = {l : np.log(price_vect[l+1]) - np.log(price_vect[l]) for l in range(n-1)}
    start_time = time.time()
    
    for k in range(-cutoff_freq, cutoff_freq + 1) :
        c = 0
        for l in range(n-1) :
            res = complex(0, - k*time_vect_res[l])
            c += (math.e ** res) * dico_delta[l]
        dico_coeffs[k] = c 
    end_time = time.time()
    return dico_coeffs, end_time - start_time

## 1.2 Vectorized Implementation Algorithm

The Fourier coefficients are calculated via a vector and matrix product. The complexity of this algorithm remains to be $O(N^{2})$, however the numpy modules handles calculations better and provides us with slightly better results in terms of speed for lower ranges of $N$.

In [ ]:
def vectorised(time_vect_res, price_vect, cutoff_freq = math.inf):
    N = min(cutoff_freq, Nyquist_freq(time_vect_res))

    n = len(time_vect_res)
    delta = [np.log(price_vect[k+1]) - np.log(price_vect[k]) for k in range(0, n-1)]
    delta = np.transpose([delta])
    start_time = time.time()

    lis = np.asarray([list(range(1,N+1))])
    c_n = np.exp(-1j*np.transpose(lis).dot(np.asarray([time_vect_res[:-1]]))).dot(delta)
    
    dico_coeffs = {}
    
    for k in range(0, N+1) :
        if k == 0 : 
            dico_coeffs[k] = np.sum(delta)
        else : 
            dico_coeffs[k] = c_n[k-1][0]
            dico_coeffs[-k] = np.conj(c_n[k-1][0])

    end_time = time.time()

    return (dico_coeffs, end_time-start_time)

## 1.3 Fast Fourier Transform

The Fast Fourier Transform is interesting to compute the Fourier coefficients since it reduces the initially quadratic complexity to $O(Nlog(N))$. However, its main drawback is that the algorithm supposes that the asset prices are observed at the exact same time, rendering it unusable for asynchronous data.

In [14]:
def fast_fourier_transform(time_vect_res, price_vect, cutoff_freq = math.inf) :
    N = min(cutoff_freq, Nyquist_freq(time_vect_res))
    
    n = len(time_vect_res)
    delta = [np.log(price_vect[k+1]) - np.log(price_vect[k]) for k in range(0, n-1)]
    
    start_time = time.time()
    res = fft(delta, min(2*N, n))
    end_time = time.time()
    dic = {}
    frequencies = fftfreq(len(res), 1/len(res))
    for i in range(len(res)) : 
        dic[int(frequencies[i])] = res[i]
    return dic, end_time - start_time

## 1.4 Zero Padded Fast Fourier Transform

The Zero Padded FFT aims to solve the problem posed by asynchrony by zero padding the observation times of every single asset. This produces seemingly synchronous asset price vectors, which makes it possible to use the Fast Fourier Transform and benefit from its $O(Nlog(N))$ complexity. However this method's drawback is its poor precision.

In [6]:
def zero_padded_fft(time_vect_res, price_vect) :
    n = len(time_vect_res)
    
    step = time_vect_res[1] - time_vect_res[0]
    for i in range(n-1) :
        if time_vect_res[i+1] - time_vect_res[i] < step :
            step = time_vect_res[i+1] - time_vect_res[i]
    
    delta = [np.log(price_vect[k+1]) - np.log(price_vect[k]) for k in range(0, n-1)]
    expected_length = math.floor(2*np.pi/step) + 1
    start_time = time.time()
    new_delta = [0 for i in range(expected_length)]
    for j in range(n-1) :
        l = math.floor(expected_length*time_vect_res[j]/(2*np.pi))
        new_delta[l] = delta[j]
    
    result = fft(new_delta)
    end_time = time.time()
    
    dic = {}
    frequencies = fftfreq(len(result), 1/len(result))
    for i in range(len(result)) : 
        dic[int(frequencies[i])] = result[i]
    
    return dic, end_time - start_time

# 2. Non Uniform Fourier Transform Algorithms

The non uniform Fourier Transform is an extension of the general definition of the Fourier transform where the observation points (x_{0},...,x_{N-1}) are not necessarily sampled in a uniform way. The Fourier coefficient of the frequency $k \in \mathbb{K} \subset \mathbb{N}$ is defined by : 
\begin{equation*}
    F(\textbf{k}) := \sum_{j=0}^{N-1} f_{j}e^{-i\textbf{k}.\textbf{x}_{j}} \quad \text{pour} \quad \textbf{k} \in K.
\end{equation*}
We can notice that :
\begin{equation*}
    F(k)=\sum_{n=0}^{N-1} f_{n} e^{-i k x_{n}}=\int\left(\sum_{n=0}^{N-1} f_{n} \delta\left(t-x_{n}\right)\right) e^{-i k t} d t=G(k).
\end{equation*}
where $G$ is the Fourier transform of the function $g(x) := \sum_{j=0}^{N-1} f_{j}\delta(x-x_{j})$. <br>
The main idea behind the non uniform Fourier transform is to convolve the function $g$ by a sufficiently regular function, called kernel, satisfying certain properties, that make it possible to over-sample the convoluted product on a regular grid of length $M = \sigma N$ and perform the Fast Fourier Transform on it. The Fourier transform F is finally obtained by deconvolving (which is equivalent to dividing by the Fourier transform of the chosen kernel). The parameter $\sigma$ is usually taken equal to 2. The algorithms involve more parameters that are fine tuned to increase performance and accuracy. <br>
For a more detailed overview about the non uniform Fourier transform, an excerpt of that section on our final report (in French) is available in the repositery. 

We will now implement the non uniform Fourier transform using three popular kernels.

## 2.1 Non Uniform Fast Fourier Transform : Exponential of semi-circle kernel

\begin{equation*}
    \phi_{E S}(x)=\left\{\begin{array}{ll}
e^{\beta \sqrt{1-x^{2}}-1} & \text{si }|x| \leq 1, \\
0 & \text {sinon}.
\end{array}\right.
\end{equation*}

In [18]:
def ES(z, w) :
    b = 2.3*w
    if abs(z) <= 1 :
        return exp(b*(math.sqrt(1-z**2) - 1))
    return 0

def ES_kernel(x, w, Mr) :
    a = np.pi *w / Mr
    return ES(x/a, w)

def Fourier_ES_kernel(k, w, Mr) :
    a = np.pi * w / Mr
    def f(x) :
        return a*np.exp(complex(0, a*k*x))*ES(x, w)
    return quad(f, -1, 1)[0]

In [8]:
def NUFFT_ES(time_vect_res, price_vect, M, tol) :
    """
    time_vect_res : rescaled time_vector
    price_vect : price vector of a product with respect to the time vector not log(price)
    tol : desired precision
    M : number of Fourier modes to be computed 
    """
    
    n = len(time_vect_res)
    
    dp = [np.log(price_vect[i+1])-np.log(price_vect[i]) for i in range(n-1)] #différentielle des log-prix
    
    start_time = time.time()
    Mr = sigma*M
    Msp = math.floor(0.5*(2-np.log10(tol))+2)
    w = 2*Msp + 1
    h = 2*np.pi/Mr
    
    over_sample = [0 for i in range(Mr)] 
    
    for j in range(n-1) :
        xj = time_vect_res[j]
        xhi = math.floor(xj/h)
        diff = xj - xhi*h
        
        for k in range(-Msp, Msp+1) :
            i = xhi - k
            if i > Mr - 1 :
                i = i%Mr
            z = dp[j]*ES_kernel(diff - k*h, w, Mr)
            over_sample[i] = over_sample[i] + z
        
    fourier_coeffs = fft(over_sample)
    frequencies = fftfreq(len(fourier_coeffs), 1/len(fourier_coeffs))
    freqs = list(frequencies[:int((M-1)/2)+1]) + list(frequencies[-int((M-1)/2):])
    
    dic = {}
    l = len(fourier_coeffs)
    
    for k in freqs : 
        dic[int(k)] = 2*np.pi*fourier_coeffs[int(k)]/(Fourier_ES_kernel(int(k), w, Mr)*l)
        
    end_time = time.time()
    return dic, end_time - start_time

## 2.2 Non Uniform Fast Fourier Transform : Gaussian Kernel

\begin{equation*}
    \varphi_{G}(x)=e^{-x^{2} / 4 \tau} \text{ for } x \in \mathbb{R}, \quad \tau=\frac{1}{M^{2}} \frac{\pi}{\sigma(\sigma-0.5)} M_{s p}.
\end{equation*}

In [10]:
def GS_kernel(x, M, Msp) :
    tau = np.pi*Msp/((M**2)*sigma*(sigma-0.5))
    return np.exp(-(x**2)/(4*tau))

def Fourier_GS_kernel(k, M, Msp) :
    tau = np.pi*Msp/((M**2)*sigma*(sigma-0.5))
    return 2 * math.sqrt(np.pi * tau)*exp(-k*k*tau)

In [9]:
def NUFFT_GS(time_vect_res, price_vect, M, tol) :
    """
    time_vect_res : rescaled time_vector
    price_vect : price vector of a product with respect to the time vector not log(price)
    tol : desired precision
    M : number of Fourier modes to be computed 
    """
    
    n = len(time_vect_res)
    
    dp = [np.log(price_vect[i+1])-np.log(price_vect[i]) for i in range(n-1)] # différentielle des log-prix
    
    start_time = time.time()
    Mr = sigma*M
    Msp = math.floor(0.5 - np.log(tol)*(sigma-0.5)/(np.pi*(sigma-1)))
    h = 2*np.pi/Mr
    
    over_sample = [0 for i in range(Mr)] 
    
    for j in range(n-1) :
        xj = time_vect_res[j]
        xhi = math.floor(xj/h)
        diff = xj - xhi*h
        
        for k in range(-Msp, Msp+1) :
            i = xhi - k
            if i > Mr - 1 :
                i = i%Mr
            z = dp[j]*GS_kernel(diff - k*h, M, Msp)
            over_sample[i] = over_sample[i] + z
        
    fourier_coeffs = fft(over_sample)
    fourier_coeffs /= len(fourier_coeffs)
    frequencies = fftfreq(len(fourier_coeffs), 1/len(fourier_coeffs))
    freqs = list(frequencies[:int((M-1)/2)+1]) + list(frequencies[-int((M-1)/2):])
    
    dic = {}
    
    for k in freqs : 
        dic[int(k)] = fourier_coeffs[int(k)]/Fourier_GS_kernel(int(k), M, Msp) * (2*np.pi)
        
    end_time = time.time()
    return dic, end_time - start_time

## 2.3 Non Uniform Fast Fourier Transform : Kaiser-Bessel Kernel

\begin{equation*}
    \varphi_{K B}(x)=\frac{1}{\pi}\left\{\begin{array}{ll}
\frac{\sinh \left(b \sqrt{M_{s p}^{2}-M_{r}^{2} x^{2}}\right)}{\sqrt{M_{s p}^{2}-M_{r}^{2} x^{2}}} & \text{if }|x| \leq \frac{M_{s p}}{M_{r}}, \\
\frac{\sin \left(b \sqrt{M_{r}^{2} x^{2}-M_{s p}^{2}}\right)}{\sqrt{M_{r}^{2} x^{2}-M_{s p}^{2}}} & \text {else}.
\end{array}\right.
\end{equation*}
\begin{equation*}
    \hat{\varphi}_{K B}(k)=\frac{1}{M_{r}} I_{0}\left(m \sqrt{b^{2}-\left(2 \pi k / M_{r}\right)^{2}}\right),
\end{equation*}

In [12]:
def KB_kernel(x, Msp, Mr) :
    b = (2-1/sigma)*np.pi
    if abs(x) < Msp/Mr :
        result = np.sinh(b*math.sqrt(Msp**2 - (Mr**2)*(x**2)))/math.sqrt(Msp**2) - (Mr**2)*(x**2)
    elif abs(x) > Msp/Mr  : 
        result = np.sin(b*math.sqrt((Mr**2)*(x**2) - Msp**2))/math.sqrt((Mr**2)*(x**2) - Msp**2)
    else : 
        result = b
    return result/np.pi

def Fourier_KB_kernel(k, Msp, Mr) :
    b = (2-1/sigma)*np.pi
    result = math.sqrt(b**2 - (2*np.pi*k/Mr)**2)
    return np.i0(Msp*result)/Mr

In [15]:
def NUFFT_KB(time_vect_res, price_vect, M, tol) :
    """
    time_vect_res : rescaled time_vector
    price_vect : price vector of a product with respect to the time vector not log(price)
    tol : desired precision
    M : number of Fourier modes to be computed 
    """
    
    n = len(time_vect_res)
    
    dp = [np.log(price_vect[i+1])-np.log(price_vect[i]) for i in range(n-1)]
    
    start_time = time.time()
    Mr = sigma*M
    Msp = math.floor(0.5*(2-np.ceil(np.log10(tol))))
    h = 1/Mr
    
    over_sample = [0 for i in range(Mr)] 
    
    for j in range(n-1) :
        xj = time_vect_res[j]
        xhi = math.floor(xj/h)
        diff = xj - xhi*h
        
        for k in range(-Msp, Msp+1) :
            i = xhi - k
            if i > Mr - 1 :
                i = i%Mr
            z = dp[j]*KB_kernel(diff - k*h, Msp, Mr)
            over_sample[i] = over_sample[i] + z
        
    fourier_coeffs = fft(over_sample)
    fourier_coeffs /= len(fourier_coeffs)
    frequencies = fftfreq(len(fourier_coeffs), 1/len(fourier_coeffs))
    freqs = list(frequencies[:int((M-1)/2)+1]) + list(frequencies[-int((M-1)/2):])
    
    dic = {}
    
    for k in freqs : 
        dic[int(k)] = fourier_coeffs[int(k)]/Fourier_KB_kernel(int(k), Msp, Mr)
    end_time = time.time()
    return dic, end_time - start_time

# 3. Covariance matrix reconstruction

As proved by Malliavin and Mancino in their article [A Fourier transform for Non Parametric Estimation of Multivariate Volatility](https://projecteuclid.org/journals/annals-of-statistics/volume-37/issue-4/A-Fourier-transform-method-for-nonparametric-estimation-of-multivariate-volatility/10.1214/08-AOS633.f), the Fourier coefficients of the covariance matrix can be reconstructed from the Fourier coefficients of the log-prices by a Bohr convolution. The covariance matrix can later be recovered by a Dirichlet or a Fejer inversion.

In [2]:
def time_rescale(time_vect, p) :
    """Rescales a time vector from [0 T] to [0 p]
       time_vect : time vector to rescale, sorted in ascending order
       p : period
    """
    return [p*(time_vect[i]-time_vect[0])/(time_vect[-1]-time_vect[0]) for i in range(len(time_vect))]

def Nyquist_freq(time_vect_res) :
    """Computes the Nyquist frequency for a rescaled time vector
       time_vect_res : rescaled time vector, sorted in ascending order
    """
    dt = math.inf
    for i in range(len(time_vect_res)-1) :
        res = time_vect_res[i+1] - time_vect_res[i]
        if res < dt :
            dt = res
    return math.floor(math.pi/dt)

def covariance_matrix(prices, time_vect, method='loop', tol=1e-6, reconstruct='FEJ') :
    """ Returns a dictionary of dictionaries
       prices : a list of price vectors
       method : the name of the method to compute the coefficients
    """
    correction = {}
    for i in range(len(new_times)) :
        correction[i] = new_times[i][-1] - new_times[i][0]
        
    d = {}
    t = 0
    tmax = time_vect[-1]
    tmin = time_vect[0]
    
    for i in range(len(prices)) :
        if method == 'loop' :
            time_vect_res = time_rescale(time_vect, 2*np.pi)
            d[i], ti = loop(time_vect_res, prices[i], cutoff_freq = math.inf)  
            t += ti
        if method == 'vectorized' :
            time_vect_res = time_rescale(time_vect, 2*np.pi)
            d[i], ti = vectorised(time_vect_res, prices[i], cutoff_freq = math.inf)
            t += ti
        if method == 'fft' :
            time_vect_res = time_rescale(time_vect, 2*np.pi)
            d[i], ti = fast_fourier_transform(time_vect_res, prices[i], cutoff_freq = math.inf)
            t += ti
        if method == 'zfft' :
            time_vect_res = time_rescale(time_vect, 2*np.pi)
            d[i], ti = zero_padded_fft(time_vect_res, prices[i])
            t += ti
        if method == 'es-nufft' :
            time_vect_res = time_rescale(time_vect, 2*np.pi)
            N = Nyquist_freq(time_vect_res)
            d[i], ti = NUFFT_ES(time_vect_res, prices[i], 2*N+1, tol)
            t += ti
        if method == 'gs-nufft' :
            time_vect_res = time_rescale(time_vect, 2*np.pi)
            N = Nyquist_freq(time_vect_res)
            d[i], ti = NUFFT_GS(time_vect_res, prices[i], 2*N+1, tol)
            t += ti
        if method == 'kb-nufft' :
            time_vect_res = time_rescale(time_vect, 1)
            N = Nyquist_freq(time_vect_res)
            d[i], ti = NUFFT_KB(time_vect_res, prices[i], 2*N+1, tol)
            t += ti
        if method == 'finufft' :
            time_vect_res = time_rescale(time_vect, 2*np.pi)
            d[i], ti = native_nufft(time_vect_res, prices[i], cutoff_freq = math.inf, tol=tol)
            
    for i in d.keys() :
        for k in d[i].keys() :
            d[i][k] = d[i][k]/np.sqrt(correction[i])
    
    if reconstruct == 'FEJ' :
        return int_covol_FEJ(d), t
    elif reconstruct == 'DIR' :
        return int_covol_DIR(d), t

def int_covol_DIR(fourier_coeffs) :
    """ Returns the estimator of the integrated covariance matrix
       fourier_coeffs : dictionnary containing the fourier coefficients of each asset
    """
    nbr_of_assets = len(fourier_coeffs)
    covol = [[] for i in range(nbr_of_assets)]
    N = int((len(fourier_coeffs[0])-1)/2)
    for i in range(nbr_of_assets) :
        for j in range(nbr_of_assets) :
            corr = 0
            for s in range(-N, N+1) :
                corr += fourier_coeffs[i][s]*fourier_coeffs[j][-s]
            covol[i].append((np.real(corr)/(2*N+1)))
    return covol

def int_covol_FEJ(fourier_coeffs) :
    """ Returns the estimator of the integrated covariance matrix
       fourier_coeffs : dictionnary containing the fourier coefficients of each asset
    """
    nbr_of_assets = len(fourier_coeffs)
    covol = [[] for i in range(nbr_of_assets)]
    N = int((len(fourier_coeffs[0])-1)/2)
    for i in range(nbr_of_assets) :
        for j in range(nbr_of_assets) :
            corr = 0
            for s in range(-N, N+1) :
                corr += fourier_coeffs[i][s]*fourier_coeffs[j][-s]*(1-abs(s)/N)
            covol[i].append((np.real(corr)/(N+1)))
    return covol 